# word2vec implementation

## Original papers
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)
* [word2vec Explained: deriving Mikolov et al.'s negative-sampling word-embedding method](https://arxiv.org/abs/1402.3722)



## Overview

The original paper proposed using:
1. Matmul to extract word vectors from ```Win```.
2. Softmax to calculate scores with all the word vectors from ```Wout```.

<img src="image/word2vec_cbow_mechanism.png" align="left"/>

## Negative Sampling

### Motivation

The original implementation was computationally expensive.

1. Matmal to extract word vectors from ```Win```.
2. Softmax can happen vocabulary size times with ```Wout```.

<img src="image/word2vec_negative_sampling_motivation.png" align="left"/>


### Solution

1. Use index to extract word vectors from Win.
2. Instead of calculating softmax with all the word vectors from ```Wout```, sample small number (SL) of negative/false word vectors from ```Wout``` and calculate logistic log loss with each sample. 

<img src="image/wors2vec_neg_sample_backprop.png" align="left"/>

## Using only one Word vector space W

There is no reasoning nor proof why two word vector space are required. In the end, we only use one word vector space, which appears to be ```Win```. 

However, if we use one vector space ```W``` for ```event```, ```context``` and ```negative samples```,then an event vector ```event=W[i]``` in a sentence can be used as a negative sample in another setence. Then the weight ```W[i]``` is updated for both positive and negative labels in the same gradient descent on ```W```. The actual [experiment of using only one vector space](./layer/embedding_single_vector_space.py) ```W``` did not work well.

* [Why do we need 2 matrices for word2vec or GloVe](https://datascience.stackexchange.com/a/94422/68313)


<img src="image/word2vec_why_not_one_W.png" align="left" width=800/>

---
# Setups

In [1]:
import cProfile
import sys
import os
import time
import re
from itertools import islice
from typing import Dict, List
import numpy as np
import tensorflow as tf

np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=400) 

## Setup for Google Colab environment

Colab gets disconnected within approx 20 min. Hence not suitable for training (or need to upgrade to the pro version).

### Clone github to Google Drive

In [2]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/drive/MyDrive/github
    !mkdir -p /content/drive/MyDrive/github
    !git clone https://github.com/oonisim/python-programs.git /content/drive/MyDrive/github


### Clone github to local directory

In [3]:
try:
    import google.colab
    IN_GOOGLE_COLAB = True
except:
    IN_GOOGLE_COLAB = False
    
if IN_GOOGLE_COLAB:
    !pip install line_profiler
    !google.colab.drive.mount('/content/gdrive')
    !rm -rf /content/github
    !mkdir -p /content/github
    !git clone https://github.com/oonisim/python-programs.git /content/github
        
    import sys
    sys.path.append('/content/github/nlp/src')

# Jupyter notebook setups

Auto reolaod causes an error in Jupyter notebooks. Restart the Jupyter kernel for the error:
```TypeError: super(type, obj): obj must be an instance or subtype of type```
See
- https://stackoverflow.com/a/52927102/4281353
- http://thomas-cokelaer.info/blog/2011/09/382/

> The problem resides in the mechanism of reloading modules.
> Reloading a module often changes the internal object in memory which
> makes the isinstance test of super return False.

In [4]:
%load_ext line_profiler
%load_ext autoreload

## Utilites

In [5]:
%autoreload 2

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

import function.fileio as fileio
import function.text as text

---
# Data Types


In [6]:
from common.constant import (
    TYPE_INT,
    TYPE_FLOAT,
    TYPE_LABEL,
    TYPE_TENSOR,
)

# Constants

Too large LR generates unusable event vector space.

Uniform weight distribution does not work (Why?)
Weights from the normal distribution sampling with small std (0.01) works (Why?)

In [7]:
USE_TEXT8 = False
USE_PTB = not USE_TEXT8
USE_CBOW = False
USE_SGRAM = not USE_CBOW

CORPUS_FILE = "text8_256" if USE_TEXT8 else "ptb_train"
CORPUS_URL = "https://data.deepai.org/text8.zip" \
    if USE_TEXT8 else f'https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt' \

TARGET_SIZE = TYPE_INT(1)       # Size of the target event (word)
CONTEXT_SIZE = TYPE_INT(10)     # Size of the context in which the target event occurs.
WINDOW_SIZE = TARGET_SIZE + CONTEXT_SIZE
SAMPLE_SIZE = TYPE_INT(5)       # Size of the negative samples

VECTOR_SIZE = TYPE_INT(100)     # Number of features in the event vector.
WEIGHT_SCHEME = "normal"
WEIGHT_PARAMS = {
    "std": 0.01
}

LR = TYPE_FLOAT(20)
NUM_SENTENCES = 10

STATE_FILE = "../models/word2vec_sgram_%s_E%s_C%s_S%s_W%s_%s_%s_V%s_LR%s_N%s.pkl" % (
    CORPUS_FILE,
    TARGET_SIZE,
    CONTEXT_SIZE,
    SAMPLE_SIZE,
    WEIGHT_SCHEME,
    "std",
    WEIGHT_PARAMS["std"],
    VECTOR_SIZE,
    LR,
    NUM_SENTENCES,
)

MAX_ITERATIONS = 100000

---

# Data
## Corpus

In [8]:
path_to_corpus = f"~/.keras/datasets/{CORPUS_FILE}"
if fileio.Function.is_file(path_to_corpus):
    pass
else:
    # text8, run "cat text8 | xargs -n 512 > text8_512" after download
    path_to_corpus = tf.keras.utils.get_file(
        fname=CORPUS_FILE,
        origin=CORPUS_URL,
        extract=True
    )
corpus = fileio.Function.read_file(path_to_corpus)
print(path_to_corpus)

/home/oonisim/.keras/datasets/ptb_train


In [9]:
examples = corpus.split('\n')[:1]
for line in examples:
    print(line)

 aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter 


---
# Event (word) indexing
Index the events that have occurred in the event sequence.

In [10]:
%autoreload 2
from layer.preprocessing import (
    EventIndexing
)

/home/oonisim/conda/envs/python_programs/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [11]:
word_indexing = EventIndexing(
    name="word_indexing",
    corpus=corpus,
    min_sequence_length=WINDOW_SIZE
)
del corpus

## EventIndexing  for the corpus

Adapt to the ```corpus``` and provides:
* event_to_index dictionary
* vocaburary of the corpus
* word occurrence probabilites

In [12]:
words = word_indexing.list_events(range(10))
print(f"EventIndexing.vocabulary[10]:\n{words}\n")

indices = word_indexing.list_indices(words)
print(f"EventIndexing.event_to_index[10]:")
for item in zip(words, indices):
    print(item)

probabilities = word_indexing.list_probabilities(words)
print(f"\nEventIndexing.probabilities[10]:")
for word, p in zip(words, probabilities):
    print(f"{word:20s} : {p:.5e}")

EventIndexing.vocabulary[10]:
['<nil>' '<unk>' 'aer' 'banknote' 'berlitz' 'calloway' 'centrust' 'cluett' 'fromstein' 'gitano']

EventIndexing.event_to_index[10]:
('<nil>', 0)
('<unk>', 1)
('aer', 2)
('banknote', 3)
('berlitz', 4)
('calloway', 5)
('centrust', 6)
('cluett', 7)
('fromstein', 8)
('gitano', 9)

EventIndexing.probabilities[10]:
<nil>                : 0.00000e+00
<unk>                : 1.65308e-02
aer                  : 5.34860e-06
banknote             : 5.34860e-06
berlitz              : 5.34860e-06
calloway             : 5.34860e-06
centrust             : 5.34860e-06
cluett               : 5.34860e-06
fromstein            : 5.34860e-06
gitano               : 5.34860e-06


## Sampling using the probability

Sample events according to their probabilities.

In [13]:
sample = word_indexing.sample(size=5)
print(sample)

['addressing', 'abroad', 'carefully', 'maryland', 'co']


## Negative Sampling
Sample events not including those events already sampled.

In [14]:
negative_indices = word_indexing.negative_sample_indices(
    size=5, excludes=word_indexing.list_indices(sample)
)
print(f"negative_indices={negative_indices} \nevents={word_indexing.list_events(negative_indices)}")

negative_indices=[992, 36, 3110, 44, 237] 
events=['addresses' 'as' 'tissue' 'of' 'argue']


## Sentence to Sequence

In [15]:
# sentences = "\n".join(corpus.split('\n')[5:6])
sentences = """
the asbestos fiber <unk> is unusually <unk> once it enters the <unk> 
"""
sequences = word_indexing.function(sentences)
for pair in zip(sentences.strip().split(" "), sequences[0]):
    print(f"{pair[0]:15} : {pair[1]:5}")

Sentence is empty. Skipping...
Sentence is empty. Skipping...


the             :    34
asbestos        :    63
fiber           :    86
<unk>           :     1
is              :    42
unusually       :    87
<unk>           :     1
once            :    64
it              :    80
enters          :    88
the             :    34
<unk>           :     1


---
# EventContext

EventContext layer generates ```(event, context)``` pairs from sequences.

In [16]:
%autoreload 2
from layer.preprocessing import (
    EventContext
)

In [17]:
event_context = EventContext(
    name="ev",
    window_size=WINDOW_SIZE,
    event_size=TARGET_SIZE
)

## Context of an event (word) in a sentence

In the sentence ```"a form of asbestos once used to make kent cigarette filters"```, one of the context windows ```a form of asbestos once``` of size 5 and event size 1 has.
* ```of``` as a target word.
* ```(a, form) and (asbestos, once)``` as its context.

### Sequence of the word indices for the sentence

In [18]:
sentences = """
a form of asbestos once used to make kent cigarette filters

N years old and former chairman of consolidated gold 
"""

sequence = word_indexing.function(sentences)
sequence

Sentence is empty. Skipping...
Sentence is empty. Skipping...
Sentence is empty. Skipping...


array([[37, 62, 44, 63, 64, 65, 66, 67, 68, 69, 70],
       [29, 30, 31, 50, 51, 43, 44, 52, 53,  0,  0]])

## (event, context) pairs

For each word (event) in the setence ```(of, asbestos, ... , kent)``` excludnig the ends of the sentence, create ```(target, context)``` as:

```
[
  [of, a, form, asbestos, once],              # target is 'of', context is (a, form, asbestos, once)
  ['asbestos', 'form', 'of', 'once', 'used'],
  ['once', 'of', 'asbestos', 'used', 'to'],
  ...
]
```

### Format of the (event, context) pairs

* **E** is the target event indices
* **C** is the context indices

<img src="image/event_context_format.png" align="left"/>

In [19]:
event_context_pairs = event_context.function(sequence)
print(
    f"Event context pairs. Shape %s, Target event size %s, Window size %s." % 
    (event_context_pairs.shape, event_context.event_size, event_context.window_size)
)
event_context_pairs[:10]

Event context pairs. Shape (2, 11), Target event size 1, Window size 11.


array([[65, 37, 62, 44, 63, 64, 66, 67, 68, 69, 70],
       [43, 29, 30, 31, 50, 51, 44, 52, 53,  0,  0]], dtype=int32)

### (event, context) pairs in textual words

In [20]:
word_indexing.sequence_to_sentence(event_context_pairs[:10])

[['used',
  'a',
  'form',
  'of',
  'asbestos',
  'once',
  'to',
  'make',
  'kent',
  'cigarette',
  'filters'],
 ['chairman',
  'n',
  'years',
  'old',
  'and',
  'former',
  'of',
  'consolidated',
  'gold',
  '<nil>',
  '<nil>']]

---
# Word Embedding

Embedding is to train the model to group similar events in a close proximity in the event vector space. If two events e.g. 'pencil' and 'pen' are similar concepts, then their event vectors resides in a close distance in the event space. 

* [Thought Vectors](https://wiki.pathmind.com/thought-vectors)

## Training process

1. Calculate ```Bc```, the BoW (Bag of Words) from context event vectors.
2. Calculate ```Be```,  the BoW (Bag of Words) from target event vectors.
3. The dot product ```Ye = dot(Bc, Be)``` is given the label 1 to get them closer.
4. For each negative sample ```Ws(s)```, the dot product with ```Ys = dot(Be, Ws(s)``` is given the label 0 to get them apart. 
5. ```np.c_[Ye, Ys]``` is fowarded to the logistic log loss layer.

<img src="image/word2vec_backprop_Be.png" align="left"/>


In [38]:
%autoreload 2
if USE_CBOW:
    from layer.embedding_cbow_dual_vector_spaces.py import (
        Embedding
    )
else:
    from layer.embedding_sgram import (
        Embedding
    )

from optimizer import (
    SGD
)

NameError: name 'USE_CBOW' is not defined

In [22]:
embedding: Embedding = Embedding(
    name="embedding",
    num_nodes=WINDOW_SIZE,
    target_size=TARGET_SIZE,
    context_size=CONTEXT_SIZE,
    negative_sample_size=SAMPLE_SIZE,
    event_vector_size=VECTOR_SIZE,
    optimizer=SGD(lr=LR),
    dictionary=word_indexing,
    weight_initialization_scheme=WEIGHT_SCHEME,
    weight_initialization_parameters=WEIGHT_PARAMS
)

### Scores ```np.c_[Ye, Ys]``` from the Embedding

The 0th column is the scores for ```dot(Bc, Be``` for positive labels. The rest are the scores for ```dot(Bc, Ws)``` for negative labels.

In [23]:
scores = embedding.function(event_context_pairs)
print(f"Scores:\n{scores[:5]}\n")
print(f"Scores for dot(Bc, Be): \n{scores[:5, :1]}")

Scores:
[[-9.1185665e-04 -2.4321437e-04  1.0157871e-04 -1.0421169e-03  5.6947256e-04  1.0440625e-03 -1.3973997e-03  1.0924268e-03  7.8901229e-04 -3.1144917e-04 -8.0015633e-04 -2.2692217e-04 -4.3101949e-04 -3.8034411e-04  1.9083207e-04]
 [ 5.4251461e-04  7.0913730e-04  2.2273404e-03  4.3803710e-04 -4.3168294e-04  1.2829594e-05  2.6357165e-04 -3.9755483e-05  0.0000000e+00  0.0000000e+00  4.5001745e-04 -6.9760508e-04  6.7859812e-04 -6.6164788e-04 -3.2094374e-04]]

Scores for dot(Bc, Be): 
[[-0.00091186]
 [ 0.00054251]]


---
# Batch Normalization

In [24]:
from layer.normalization import (
    BatchNormalization
)

In [25]:
bn = BatchNormalization(
    name="bn",
    num_nodes=1+SAMPLE_SIZE
)

---
# Logistic Log Loss

Train the model to get:
1. BoW of the context event vectors close to the target event vector. Label 1
2. BoW of the context event vectors away from each of the negative sample event vectors Label 0.

This is a binary logistic classification, hence use Logistic Log Loss as the network objective function.

In [26]:
from common.function import (
    sigmoid_cross_entropy_log_loss,
    sigmoid
)
from layer.objective import (
    CrossEntropyLogLoss
)

In [27]:
loss = CrossEntropyLogLoss(
    name="loss",
    num_nodes=1,  # Logistic log loss
    log_loss_function=sigmoid_cross_entropy_log_loss
)

---
# Adapter

The logistic log loss layer expects the input of shape ```(N,M=1)```, however Embedding outputs ```(N,(1+SL)``` where ```SL``` is SAMPLE_SIZE. The ```Adapter``` layer bridges between Embedding and the Loss layers.


In [28]:
from layer.adapter import (
    Adapter
)

In [29]:
adapter_function = embedding.adapt_function_to_logistic_log_loss(loss=loss)
adapter_gradient = embedding.adapt_gradient_to_logistic_log_loss()

adapter: Adapter = Adapter(
    name="adapter",
    num_nodes=1,    # Number of output M=1 
    function=adapter_function,
    gradient=adapter_gradient
)

---
# Word2vec Network

## Construct a sequential network

$ \text {Sentences} \rightarrow EventIndexing \rightarrow EventContext \rightarrow  Embedding \rightarrow Adapter \rightarrow LogisticLogLoss$

In [30]:
from network.sequential import (
    SequentialNetwork
)

In [31]:
network = SequentialNetwork(
    name="word2vec",
    num_nodes=1,
    inference_layers=[
        word_indexing,
        event_context,
        embedding,
        adapter
    ],
    objective_layers=[
        loss
    ]
)

## Run training

### Load the saved model file if exists

In [32]:
if fileio.Function.is_file(STATE_FILE):
    print("Loading model...\nSTATE_FILE: %s" % STATE_FILE)
    state = embedding.load(STATE_FILE)

    fmt="""Model loaded.
    event_size %s
    context_size: %s
    event_vector_size: %s
    """
    print(fmt % (
        state["target_size"], 
        state["context_size"], 
        state["event_vector_size"]
    ))
else:
    print("State file does not exist. Saving the initial model to %s." % STATE_FILE)
    embedding.save(STATE_FILE)

State file does not exist. Saving the initial model to ../models/word2vec_sgram_ptb_train_E1_C10_S5_Wnormal_std_0.01_V100_LR20.0_N10.pkl.


### Continue training

In [33]:
def sentences_generator(path_to_file, num_sentences):
    stream = fileio.Function.file_line_stream(path_to_file)
    try:
        while True:
            _lines = fileio.Function.take(num_sentences, stream)
            yield np.array(_lines)
    finally:
        stream.close()

# Sentences for the trainig
source = sentences_generator(
    path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
)

# Restore the state if exists.
state = embedding.load(STATE_FILE)

# Continue training
profiler = cProfile.Profile()
profiler.enable()

total_sentences = 0
epochs = 0

for i in range(MAX_ITERATIONS):
    try:
        sentences = next(source)
        total_sentences += len(sentences)

        start = time.time()
        network.train(X=sentences, T=np.array([0]))

        if i % 100 == 0:
            print(
                f"Batch {i:05d} of {NUM_SENTENCES} sentences: "
                f"Average Loss: {np.mean(network.history):10f} "
                f"Duration {time.time() - start:3f}"
            )
        if i % 1000 == 0:
            # embedding.save(STATE_FILE)
            pass
        
    except fileio.Function.GenearatorHasNoMore as e:
        source.close()
        embedding.save(STATE_FILE)

        # Next epoch
        print(f"epoch {epochs} batches {i:05d} done")
        epochs += 1
        source = sentences_generator(
            path_to_file=path_to_corpus, num_sentences=NUM_SENTENCES
        )

    except Exception as e:
        print("Unexpected error:", sys.exc_info()[0])
        source.close()
        raise e

embedding.save(STATE_FILE)

profiler.disable()
profiler.print_stats(sort="cumtime")

Batch 00000 of 10 sentences: Average Loss:   0.693137 Duration 0.211343
Batch 00100 of 10 sentences: Average Loss:   0.668239 Duration 0.283479
Batch 00200 of 10 sentences: Average Loss:   0.636050 Duration 0.192648
Batch 00300 of 10 sentences: Average Loss:   0.612916 Duration 0.420230
Batch 00400 of 10 sentences: Average Loss:   0.594639 Duration 0.183814
Batch 00500 of 10 sentences: Average Loss:   0.581592 Duration 0.376362
Batch 00600 of 10 sentences: Average Loss:   0.572798 Duration 0.172752
Batch 00700 of 10 sentences: Average Loss:   0.563167 Duration 0.281529
Batch 00800 of 10 sentences: Average Loss:   0.555399 Duration 0.259492
Batch 00900 of 10 sentences: Average Loss:   0.549399 Duration 0.272705
Batch 01000 of 10 sentences: Average Loss:   0.543852 Duration 0.407641
Batch 01100 of 10 sentences: Average Loss:   0.539172 Duration 0.289123
Batch 01200 of 10 sentences: Average Loss:   0.534918 Duration 0.254536
Batch 01300 of 10 sentences: Average Loss:   0.530460 Duration 0

Batch 11400 of 10 sentences: Average Loss:   0.439596 Duration 0.480985
Batch 11500 of 10 sentences: Average Loss:   0.439370 Duration 0.218554
Batch 11600 of 10 sentences: Average Loss:   0.439104 Duration 0.298814
Batch 11700 of 10 sentences: Average Loss:   0.438841 Duration 0.171770
Batch 11800 of 10 sentences: Average Loss:   0.438570 Duration 0.321132
Batch 11900 of 10 sentences: Average Loss:   0.438357 Duration 0.389374
Batch 12000 of 10 sentences: Average Loss:   0.438090 Duration 0.420473
Batch 12100 of 10 sentences: Average Loss:   0.437911 Duration 0.277806
Batch 12200 of 10 sentences: Average Loss:   0.437687 Duration 0.134432
Batch 12300 of 10 sentences: Average Loss:   0.437359 Duration 0.167764
Batch 12400 of 10 sentences: Average Loss:   0.437063 Duration 0.261726
Batch 12500 of 10 sentences: Average Loss:   0.436828 Duration 0.289682
Batch 12600 of 10 sentences: Average Loss:   0.436605 Duration 0.437866
epoch 2 batches 12623 done
Batch 12700 of 10 sentences: Average 

Batch 22700 of 10 sentences: Average Loss:   0.421626 Duration 0.217629
Batch 22800 of 10 sentences: Average Loss:   0.421561 Duration 0.479828
Batch 22900 of 10 sentences: Average Loss:   0.421454 Duration 0.273519
Batch 23000 of 10 sentences: Average Loss:   0.421329 Duration 0.290141
Batch 23100 of 10 sentences: Average Loss:   0.421221 Duration 0.180151
Batch 23200 of 10 sentences: Average Loss:   0.421106 Duration 0.665356
Batch 23300 of 10 sentences: Average Loss:   0.420968 Duration 0.357016
Batch 23400 of 10 sentences: Average Loss:   0.420815 Duration 0.172357
Batch 23500 of 10 sentences: Average Loss:   0.420732 Duration 0.251593
Batch 23600 of 10 sentences: Average Loss:   0.420615 Duration 0.288549
Batch 23700 of 10 sentences: Average Loss:   0.420503 Duration 0.397693
Batch 23800 of 10 sentences: Average Loss:   0.420421 Duration 0.261572
Batch 23900 of 10 sentences: Average Loss:   0.420318 Duration 0.304776
Batch 24000 of 10 sentences: Average Loss:   0.420210 Duration 0

Batch 34000 of 10 sentences: Average Loss:   0.411990 Duration 0.232713
Batch 34100 of 10 sentences: Average Loss:   0.411904 Duration 0.231471
Batch 34200 of 10 sentences: Average Loss:   0.411857 Duration 0.372552
Batch 34300 of 10 sentences: Average Loss:   0.411814 Duration 0.211706
Batch 34400 of 10 sentences: Average Loss:   0.411731 Duration 0.145076
Batch 34500 of 10 sentences: Average Loss:   0.411666 Duration 0.336111
Batch 34600 of 10 sentences: Average Loss:   0.411614 Duration 0.272889
Batch 34700 of 10 sentences: Average Loss:   0.411558 Duration 0.234218
Batch 34800 of 10 sentences: Average Loss:   0.411529 Duration 0.163997
Batch 34900 of 10 sentences: Average Loss:   0.411485 Duration 0.169789
Batch 35000 of 10 sentences: Average Loss:   0.411413 Duration 0.199089
Batch 35100 of 10 sentences: Average Loss:   0.411337 Duration 0.256553
Batch 35200 of 10 sentences: Average Loss:   0.411293 Duration 0.152184
Batch 35300 of 10 sentences: Average Loss:   0.411230 Duration 0

Batch 45400 of 10 sentences: Average Loss:   0.405388 Duration 0.227461
Batch 45500 of 10 sentences: Average Loss:   0.405346 Duration 0.197183
Batch 45600 of 10 sentences: Average Loss:   0.405281 Duration 0.231802
Batch 45700 of 10 sentences: Average Loss:   0.405220 Duration 0.231430
Batch 45800 of 10 sentences: Average Loss:   0.405185 Duration 0.198672
Batch 45900 of 10 sentences: Average Loss:   0.405112 Duration 0.209959
Batch 46000 of 10 sentences: Average Loss:   0.405050 Duration 0.171040
Batch 46100 of 10 sentences: Average Loss:   0.404975 Duration 0.201131
Batch 46200 of 10 sentences: Average Loss:   0.404918 Duration 0.166828
epoch 10 batches 46287 done
Batch 46300 of 10 sentences: Average Loss:   0.404874 Duration 0.181416
Batch 46400 of 10 sentences: Average Loss:   0.404840 Duration 0.195611
Batch 46500 of 10 sentences: Average Loss:   0.404803 Duration 0.124634
Batch 46600 of 10 sentences: Average Loss:   0.404744 Duration 0.251456
Batch 46700 of 10 sentences: Average

...

Batch 79400 of 10 sentences: Average Loss:   0.392020 Duration 0.293583
Batch 79500 of 10 sentences: Average Loss:   0.391991 Duration 0.262933
Batch 79600 of 10 sentences: Average Loss:   0.391948 Duration 0.248520
Batch 79700 of 10 sentences: Average Loss:   0.391910 Duration 0.511306
Batch 79800 of 10 sentences: Average Loss:   0.391865 Duration 0.358510
Batch 79900 of 10 sentences: Average Loss:   0.391838 Duration 0.372678
epoch 18 batches 79951 done
Batch 80000 of 10 sentences: Average Loss:   0.391807 Duration 0.433802
Batch 80100 of 10 sentences: Average Loss:   0.391781 Duration 0.374878
Batch 80200 of 10 sentences: Average Loss:   0.391762 Duration 0.314393
Batch 80300 of 10 sentences: Average Loss:   0.391718 Duration 0.362284
Batch 80400 of 10 sentences: Average Loss:   0.391681 Duration 0.211045
Batch 80500 of 10 sentences: Average Loss:   0.391663 Duration 0.229786
Batch 80600 of 10 sentences: Average Loss:   0.391638 Duration 0.215662
Batch 80700 of 10 sentences: Average

Batch 90700 of 10 sentences: Average Loss:   0.388704 Duration 0.357444
Batch 90800 of 10 sentences: Average Loss:   0.388683 Duration 0.206968
Batch 90900 of 10 sentences: Average Loss:   0.388653 Duration 0.334456
Batch 91000 of 10 sentences: Average Loss:   0.388618 Duration 0.285307
Batch 91100 of 10 sentences: Average Loss:   0.388595 Duration 0.308721
Batch 91200 of 10 sentences: Average Loss:   0.388571 Duration 0.222652
Batch 91300 of 10 sentences: Average Loss:   0.388544 Duration 0.336539
Batch 91400 of 10 sentences: Average Loss:   0.388519 Duration 0.346643
Batch 91500 of 10 sentences: Average Loss:   0.388495 Duration 0.301454
Batch 91600 of 10 sentences: Average Loss:   0.388469 Duration 0.423123
Batch 91700 of 10 sentences: Average Loss:   0.388441 Duration 0.389730
Batch 91800 of 10 sentences: Average Loss:   0.388419 Duration 0.323457
Batch 91900 of 10 sentences: Average Loss:   0.388386 Duration 0.341766
Batch 92000 of 10 sentences: Average Loss:   0.388354 Duration 0

   699839    4.925    0.000  136.636    0.000 array_ops.py:200(fill)
   599862   10.162    0.000  136.342    0.000 gen_array_ops.py:8392(reshape_eager_fallback)
   999724  136.293    0.000  136.293    0.000 utility.py:84(<listcomp>)
   699839    4.511    0.000  126.065    0.000 gen_array_ops.py:3304(fill)
   299931    1.939    0.000  117.002    0.000 base.py:308(dY)
  7591829   36.568    0.000  111.235    0.000 base.py:119(tensor_shape)
  5198804    6.765    0.000  110.978    0.000 constant_op.py:299(_constant_eager_impl)
 14715076   29.113    0.000  110.223    0.000 <__array_function__ internals>:2(count_nonzero)
  2199494    4.519    0.000  107.603    0.000 trace.py:158(wrapped)
  5198804   84.920    0.000  104.213    0.000 constant_op.py:70(convert_to_eager_tensor)
 29430152   85.339    0.000  103.495    0.000 preprocess.py:114(vocabulary_size)
  2199494   17.387    0.000  103.084    0.000 ops.py:1481(convert_to_tensor)
 35403846   64.024    0.000  101.723    0.000 tensor_util.py:99

    99977    0.606    0.000    4.357    0.000 numeric.py:148(ones)
 27159920    4.257    0.000    4.257    0.000 {method 'lower' of 'str' objects}
   999724    1.306    0.000    4.237    0.000 fromnumeric.py:3199(around)
  1799586    3.879    0.000    4.230    0.000 base.py:357(logger)
   199954    2.937    0.000    4.101    0.000 embedding_sgram.py:182(target_indices)
  1599632    2.510    0.000    3.992    0.000 tensor_conversion_registry.py:114(get)
   499885    3.976    0.000    3.976    0.000 arrayprint.py:358(_get_formatdict)
  2899333    2.766    0.000    3.911    0.000 tensor_shape.py:821(rank)
   299931    1.864    0.000    3.637    0.000 dtypes.py:172(is_compatible_with)
  5898643    3.617    0.000    3.617    0.000 context.py:773(_handle)
 11697309    3.559    0.000    3.559    0.000 tensor_shape.py:249(value)
   499885    3.407    0.000    3.407    0.000 {built-in method builtins.locals}
   699839    0.906    0.000    3.389    0.000 ops.py:6817(_is_keras_symbolic_tensor)
  

       24    0.000    0.000    0.000    0.000 codecs.py:309(__init__)
       23    0.000    0.000    0.000    0.000 utility_file.py:35(__init__)
       24    0.000    0.000    0.000    0.000 __init__.py:145(_DType_reduce)
       24    0.000    0.000    0.000    0.000 {method 'rfind' of 'str' objects}
        3    0.000    0.000    0.000    0.000 {method 'extend' of 'list' objects}
        6    0.000    0.000    0.000    0.000 tensor_conversion_registry.py:135(<genexpr>)
       24    0.000    0.000    0.000    0.000 posixpath.py:41(_get_sep)
       48    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x55e106cb7ac0}
        5    0.000    0.000    0.000    0.000 contextlib.py:238(helper)
       48    0.000    0.000    0.000    0.000 {method 'lstrip' of 'str' objects}
        5    0.000    0.000    0.000    0.000 contextlib.py:108(__enter__)
       72    0.000    0.000    0.000    0.000 {built-in method posix.fspath}
       46    0.000    0.000    0.000    0.0

---
# Evaluate the vector space

Verify if the trained model, or the vector space W, has encoded the words in a way that **similar** words are close in the vector space.

* [How to measure the similarity among vectors](https://math.stackexchange.com/questions/4132458)

In [34]:
n = 10
context = "cash".split()
word_indices = np.array(word_indexing.list_indices(context), dtype=TYPE_INT)

print(f"Words {context}")
print(f"Word indices {word_indices}")
print(f"prediction for {context}:\n{word_indexing.list_events([embedding.predict(word_indices, n)])}")

Words ['cash']
Word indices [413]
prediction for ['cash']:
[['dividends' 'exceed' 'amount' 'borrowings' 'expense' 'exceeding' 'emhart' 'liabilities' 'riskier' 'flow']
 ['borrowings' 'emhart' 'unpaid' 'sums' 'exceeding' 'riskier' 'financings' 'exceed' 'expenditures' 'beneficiaries']]


/home/oonisim/home/repository/git/oonisim/python_programs/nlp/src/layer/preprocessing/preprocess.py:216: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return self._vocabulary[list(iter(indices))]


---
## Compare with [gensim word2vec](https://radimrehurek.com/gensim/models/word2vec.html)

In [35]:
from gensim.models import (
    Word2Vec
)
from gensim.models.word2vec import (
    LineSentence    
)

In [36]:
sentences = LineSentence(source=path_to_corpus)
w2v = Word2Vec(
    sentences=sentences, 
    sg=0,
    window=5, 
    negative=5,
    vector_size=100, 
    min_count=1, 
    workers=4
)
del sentences

In [37]:
w2v.wv.most_similar(context, topn=n)

[('amount', 0.8858454823493958),
 ('debt', 0.8851679563522339),
 ('assets', 0.8694609999656677),
 ('payments', 0.8522050380706787),
 ('value', 0.8282856941223145),
 ('proceeds', 0.8225603103637695),
 ('dividend', 0.8094569444656372),
 ('reserves', 0.808117687702179),
 ('face', 0.8079915046691895),
 ('total', 0.8020584583282471)]